In [ ]:
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"

Installed Packages MathNet.Numerics, 4.15.0 MathNet.Numerics.FSharp, 4.15.0

In [ ]:
open MathNet.Numerics
open MathNet.Numerics.LinearAlgebra
open MathNet.Numerics.LinearAlgebra.Vector
open MathNet.Numerics.LinearAlgebra.Double

In [ ]:
// target function
let regressionCurve coef input = 
    coef
    |> Vector.foldi (fun i acc item -> acc + item * Math.Pow (input, double i)) 0.0

// mean squared error
// datasets [[X0, y0]; [X1, y1]; [X2, y2]]
// lsmDerivEq -> 
//   1/N * [(x_0^(i+0) + x_1^(i+0)...)a_0 + (x_0^(i+0) + x_1^(i+0)...)a_1 ... + (x_0^(i+0) + x_1^(i+0)...)a_I] 
//     = y_0 * x_0^i + y_1 * x_1^i ...   (i = coefIndex, I = coefNum)
let lsmDerivEquation coefNum dataSets =
    let dataNum = dataSets |> Matrix.rowCount
    
    // left -> X
    let eqCoefsLeft coefIndex = 
        DenseVector.init coefNum
            (fun pos -> 
                let preResult =
                    dataSets
                    |> Matrix.foldRows
                        (fun acc item -> acc + Math.Pow (item.[0], coefIndex + pos |> double))
                        0.0
                preResult / (double dataNum)
            )
        
    // right -> y
    let eqCoefRight coefIndex = 
        let preResult = 
            dataSets 
            |> Matrix.foldRows 
                (fun acc item -> acc + item.[1] * Math.Pow (item.[0], coefIndex |> double)) 
                0.0
        preResult / (double dataNum)

    let rec eqAllCoefsLeftTR num acc = 
        match num with
        | -1 -> acc
        | _ -> eqAllCoefsLeftTR (num - 1) ((eqCoefsLeft num).ToRowMatrix().Stack(acc))

    let eqAllCoefsLeft = eqAllCoefsLeftTR (coefNum - 1 - 1) ((coefNum - 1 |> eqCoefsLeft).ToRowMatrix())
    let eqAllCoefRight = DenseVector.init coefNum (fun pos -> eqCoefRight pos)
    
    (eqAllCoefsLeft, eqAllCoefRight)

// solve regression curve's coefficients
let regressionCurveCoef degree dataSets = 
    let x, y = lsmDerivEquation (degree + 1) dataSets
    x.Solve(y)

In [ ]:
let dataSets = 
    matrix [[0.000000; 0.073346];
            [0.100000; 0.663607];
            [0.200000; 0.794265];
            [0.300000; 1.045710];
            [0.400000; 0.265533];
            [0.500000; -0.452839];
            [0.600000; -0.297819];
            [0.700000; -1.061552];
            [0.800000; -0.961003];
            [0.900000; -0.248203]]

In [ ]:
let coef = regressionCurveCoef 5 dataSets
coef

index,value
0,0.05786809230886636
1,8.522437633007273
2,-22.769146211308897
3,3.379187992872957
4,13.662362473928702
5,-1.6618205141598628


In [ ]:
let regressionCurveWithCoef = regressionCurve coef

In [ ]:
#r "nuget: xplot.plotly"
#r "nuget: xplot.googlecharts"
#r "nuget: XPlot.Plotly.Interactive"

Installed Packages xplot.googlecharts, 3.0.1 xplot.plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [ ]:
open XPlot.Plotly

In [ ]:
let layout =
    Layout(
        xaxis = Xaxis(range = [-0.1; 1.1], zeroline = false),
        yaxis = Yaxis(range = [-1.1; 1.1], zeroline = false)
    )

let curve = 
    Scatter(
        x = [for x in -1.1 .. 0.01 .. 1.1 -> x],
        y = [for x in -1.1 .. 0.01 .. 1.1 -> regressionCurveWithCoef x],
        mode = "lines"
    )

let dataPoints = 
    Scatter(
        x = dataSets.Column(0),
        y = dataSets.Column(1),
        mode = "markers"
    )

let chart2 =
    [curve; dataPoints]
    |> Chart.Plot
    |> Chart.WithLayout(layout)
    |> Chart.WithHeight 500
    |> Chart.WithWidth 800
chart2

<!DOCTYPE html>